## LAB 5 [WORKING COPY] - Hyto

In [1]:
import sklearn
import matplotlib.pyplot as plt
from typing import Tuple
from typing import List
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
%matplotlib inline

import numpy as np
import pandas as pd
import scipy
import pandas as pd 
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, classification_report, precision_recall_curve, average_precision_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer #to turn categorial variables into numeric arrays
from sklearn import preprocessing #to transform the feature labels
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import mean_squared_error, r2_score

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df_2017 = pd.read_csv("../2017.csv", low_memory=False, encoding = "ISO-8859-1",  header=None)
df_2017.head()
# df_2017.reset_index()

,0,1,2,3,4,5,6,7
0,US1MNWR0029,20170101,PRCP,0,NaN,NaN,N,NaN
1,CA1MB000296,20170101,PRCP,0,NaN,NaN,N,NaN
2,ASN00015643,20170101,TMAX,274,NaN,NaN,a,NaN
3,ASN00015643,20170101,TMIN,218,NaN,NaN,a,NaN
4,ASN00015643,20170101,PRCP,2,NaN,NaN,a,NaN


In [3]:
df_2017.rename(index=int,columns = {0:'ID',1:'date', 2:'element',3:'data_value',4:'mFlag',5:'qFlag',6:'sFlag',7:'obsTime'}, inplace=True)

In [11]:
df_2017.head()

,ID,date,element,data_value,mFlag,qFlag,sFlag,obsTime
1648587,USC00242275,20170118,SNWD,0,T,I,7,800.0
5274622,USC00479304,20170224,SNWD,0,T,I,7,900.0
5487604,RQC00664614,20170226,PRCP,0,T,L,7,800.0
5752573,USC00471416,20170301,SNWD,0,T,I,7,800.0
5752579,USC00472973,20170301,SNWD,0,T,I,7,700.0


In [5]:
# column_1 = df_2017.ID.values
# column_2 = df_2017.date.values


# X = np.array(list(zip(column_1,column_2)))
# X

In [17]:
# df_2017['merged_index'] = str(df_2017['ID'])+str(df_2017['date'])

In [7]:
df_2017.isnull().sum()
df_2017 = df_2017.dropna()

In [18]:
values = ['PRCP','SNOW','SNWD','TMAX','TMIN']
# df_2017 = df_2017.iloc[df_2017['element'] == 'PRCP','SNOW','SNWD','TMAX','TMIN']
df_2017 = df_2017[df_2017['element'].isin(values)]


In [19]:
print(len(df_2017))
print(df_2017.element.unique())

26279902
['PRCP' 'TMAX' 'TMIN' 'SNOW' 'SNWD']


In [8]:
df_2017.head()

,ID,date,element,data_value,mFlag,qFlag,sFlag,obsTime
1648587,USC00242275,20170118,SNWD,0,T,I,7,800.0
5274622,USC00479304,20170224,SNWD,0,T,I,7,900.0
5487604,RQC00664614,20170226,PRCP,0,T,L,7,800.0
5752573,USC00471416,20170301,SNWD,0,T,I,7,800.0
5752579,USC00472973,20170301,SNWD,0,T,I,7,700.0


In [21]:
df_unstacked = df_2017.pivot( columns='element', values='data_value')

In [22]:
df_unstacked.head()

element,PRCP,SNOW,SNWD,TMAX,TMIN
0,0.0,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,274.0,NaN
3,NaN,NaN,NaN,NaN,218.0
4,2.0,NaN,NaN,NaN,NaN


In [10]:
df_2017['element'].unique()

array(['SNWD', 'PRCP'], dtype=object)

In [24]:
#df_2017['merged'] = df_2017[['ID', 'date']].apply(lambda x: ''.join(str(x)), axis = 0)
df_2017['merged'] = df_2017['ID'].combine_first(df_2017['date'])

In [77]:
df_idd = pd.DataFrame(df_2017[['ID', 'date']])
df_idd.head()

,ID,date
0,US1MNWR0029,20170101
1,CA1MB000296,20170101
2,ASN00015643,20170101
3,ASN00015643,20170101
4,ASN00015643,20170101


In [26]:
df_2017['merged'] = df_2017[['ID','date']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)


In [31]:
df_2017.head()

,ID,date,element,data_value,mFlag,qFlag,sFlag,obsTime,merged_index,merged
0,US1MNWR0029,20170101,PRCP,0,NaN,NaN,N,NaN,0 US1MNWR0029\n1 CA1MB0002...,US1MNWR002920170101
1,CA1MB000296,20170101,PRCP,0,NaN,NaN,N,NaN,0 US1MNWR0029\n1 CA1MB0002...,CA1MB00029620170101
2,ASN00015643,20170101,TMAX,274,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB0002...,ASN0001564320170101
3,ASN00015643,20170101,TMIN,218,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB0002...,ASN0001564320170101
4,ASN00015643,20170101,PRCP,2,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB0002...,ASN0001564320170101


In [28]:
df_unstacked = df_2017.pivot( index='merged', columns='element', values='data_value')

In [32]:
df_unstacked.head()

element,PRCP,SNOW,SNWD,TMAX,TMIN
merged,,,,,
AE00004119620170102,NaN,NaN,NaN,264.0,NaN
AE00004119620170103,NaN,NaN,NaN,NaN,156.0
AE00004119620170105,NaN,NaN,NaN,NaN,176.0
AE00004119620170107,NaN,NaN,NaN,NaN,130.0
AE00004119620170108,NaN,NaN,NaN,251.0,128.0


In [33]:
df_unstacked.isnull().sum()

element
PRCP    1069540
SNOW    6755255
SNWD    7840264
TMAX    6578286
TMIN    6550348
dtype: int64

In [35]:
len(df_unstacked)

11014719

In [41]:
linreg = LinearRegression()

In [52]:
df_unstacked = df_unstacked.dropna()

In [53]:
feature_data = df_unstacked[['PRCP','SNOW','SNWD','TMAX']]
target_data = df_unstacked[['TMIN']]

In [54]:
X, X_test, y, y_test = train_test_split(feature_data, target_data, test_size = 0.3 , random_state=100)

In [55]:
len(df_unstacked)

1463443

In [56]:
linreg = LinearRegression()
linreg = linreg.fit(X, y)


In [61]:
print(linreg.score(X,y))
print(linreg.score(X_test,y_test))

0.805397297588
0.80622495296


In [65]:
y_pred = linreg.predict(X_test)

print("Mean squared error: %.2f"
     % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
# print('Variance score: %.2f' % r2_score(y, y_pred))

Mean squared error: 2147.14
